In [1]:
#importing lib
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler

In [2]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Tensorflow version 2.2.0
Running on TPU  ['10.117.134.138:8470']
INFO:tensorflow:Initializing the TPU system: grpc://10.117.134.138:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.117.134.138:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [3]:
keras = tf.keras

In [4]:
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

In [5]:
recovered_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,6/13/20,6/14/20,6/15/20,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,4201,4725,5164,5508,6158,7660,7962,8292,8764,8841,9260,9869,10174,10306,10674,12604,13934,14131,15651,16041,17331,19164,19366,20103,20179,20700,20847,20882,21135,21216,21254,21454,22456,22824,23151,23273,23634,23741,23741,23924
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1039,1044,1055,1064,1077,1086,1114,1126,1134,1159,1195,1217,1250,1298,1346,1384,1438,1459,1516,1559,1592,1637,1657,1702,1744,1791,1832,1875,1881,1946,2014,2062,2091,2137,2214,2264,2311,2352,2397,2463
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,7420,7606,7735,7842,7943,8078,8196,8324,8422,8559,8674,8792,8920,9066,9202,9371,9674,9897,10040,10342,10832,11181,11492,11884,12094,12329,12637,13124,13124,13743,14019,14295,14792,15107,15430,15744,16051,16400,16646,16983
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,781,781,789,789,791,792,792,792,792,796,797,797,797,799,799,799,799,799,799,800,800,800,800,800,800,802,802,803,803,803,803,803,803,803,803,803,803,803,803,803
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,61,61,64,64,64,64,66,66,77,77,77,77,81,81,81,81,93,93,97,97,107,108,108,108,117,117,117,117,118,118,118,118,124,124,199,210,221,221,221,221


In [6]:
recovered_df = recovered_df.groupby('Country/Region').sum().reset_index()

In [7]:
recovered_df.head()

,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,...,6/13/20,6/14/20,6/15/20,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20
0,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,4201,4725,5164,5508,6158,7660,7962,8292,8764,8841,9260,9869,10174,10306,10674,12604,13934,14131,15651,16041,17331,19164,19366,20103,20179,20700,20847,20882,21135,21216,21254,21454,22456,22824,23151,23273,23634,23741,23741,23924
1,Albania,41.15330,20.168300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1039,1044,1055,1064,1077,1086,1114,1126,1134,1159,1195,1217,1250,1298,1346,1384,1438,1459,1516,1559,1592,1637,1657,1702,1744,1791,1832,1875,1881,1946,2014,2062,2091,2137,2214,2264,2311,2352,2397,2463
2,Algeria,28.03390,1.659600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,7420,7606,7735,7842,7943,8078,8196,8324,8422,8559,8674,8792,8920,9066,9202,9371,9674,9897,10040,10342,10832,11181,11492,11884,12094,12329,12637,13124,13124,13743,14019,14295,14792,15107,15430,15744,16051,16400,16646,16983
3,Andorra,42.50630,1.521800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,781,781,789,789,791,792,792,792,792,796,797,797,797,799,799,799,799,799,799,800,800,800,800,800,800,802,802,803,803,803,803,803,803,803,803,803,803,803,803,803
4,Angola,-11.20270,17.873900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,61,61,64,64,64,64,66,66,77,77,77,77,81,81,81,81,93,93,97,97,107,108,108,108,117,117,117,117,118,118,118,118,124,124,199,210,221,221,221,221


In [8]:
Country_recovered =  recovered_df['Country/Region'].values
recovered_df = recovered_df.drop(['Lat','Long','Country/Region'], axis=1)
recovered_df = recovered_df[recovered_df.columns].T
recovered_df.columns = Country_recovered
Date_index = recovered_df.index
recovered_df = recovered_df.diff().fillna(0)
recovered_df = recovered_df.abs()
recovered_df.head()

,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,Azerbaijan,Bahamas,Bahrain,Bangladesh,Barbados,Belarus,Belgium,Belize,Benin,Bhutan,Bolivia,Bosnia and Herzegovina,Botswana,Brazil,Brunei,Bulgaria,Burkina Faso,Burma,Burundi,Cabo Verde,Cambodia,Cameroon,Canada,Central African Republic,Chad,Chile,China,Colombia,Comoros,Congo (Brazzaville),...,Senegal,Serbia,Seychelles,Sierra Leone,Singapore,Slovakia,Slovenia,Somalia,South Africa,South Sudan,Spain,Sri Lanka,Sudan,Suriname,Sweden,Switzerland,Syria,Taiwan*,Tajikistan,Tanzania,Thailand,Timor-Leste,Togo,Trinidad and Tobago,Tunisia,Turkey,US,Uganda,Ukraine,United Arab Emirates,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
1/22/20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1/23/20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1/24/20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1/25/20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1/26/20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
#scaling the data
sc = RobustScaler()
recovered_df_scaled = sc.fit_transform(recovered_df)
recovered_df_scaled = pd.DataFrame(recovered_df_scaled)

In [10]:

DAYS_TO_PREDICT = int(input('Enter Number Of Days You Want to Predict : '))
predict_recovered_index = pd.date_range(start=Date_index[-1],periods=DAYS_TO_PREDICT + 1,closed='right')
predict_recovered_index = predict_recovered_index.strftime('%Y/%m/%d')

Enter Number Of Days You Want to Predict : 5


In [11]:
predict_recovered_index

Index(['2020/07/23', '2020/07/24', '2020/07/25', '2020/07/26', '2020/07/27'], dtype='object')

In [12]:
#creating a class for the model and the dataset
class Corona_recovered_model():
    def create_dataset(self, dataset, time_steps=1, column = 0):
        dataX, dataY = [], []
        for i in range(len(dataset)-time_steps):
          a = dataset[i:(i+time_steps), column]
          dataX.append(a)
          dataY.append(dataset[i + time_steps, column])
        return np.array(dataX), np.array(dataY)

    def Model(self,dframe,column):
        df = dframe.values
        df = df.astype('float32')
        
        time_steps = 1
        trainX, trainY = self.create_dataset(df, time_steps, column)
        trainX = np.reshape(trainX, (trainX.shape[0],1,trainX.shape[1]))
        
        
        keras.backend.clear_session()
        tf.random.set_seed(42)
        np.random.seed(42)
        recover_model = keras.models.Sequential([
            keras.layers.InputLayer(input_shape=[None, 1]),
            keras.layers.Conv1D(filters=16, kernel_size=2,
                      strides=1, padding="causal",
                      activation="relu",
                      ),
            keras.layers.LSTM(30,kernel_initializer = 'he_normal',
                              return_sequences = True),
            keras.layers.LSTM(30,kernel_initializer = 'he_normal',
                              return_sequences = True),
            keras.layers.Dense(15,kernel_initializer = 'he_normal',
                                             activation = 'relu'),
            keras.layers.Dense(15,kernel_initializer = 'he_normal',
                                             activation = 'relu'),
            keras.layers.Dense(1,activation = 'linear',kernel_initializer = 'he_normal'),
            keras.layers.Lambda(lambda x: x * 200)
             ])
        
        optimizer = keras.optimizers.Adadelta(lr = 0.001)
        
        recover_model.compile(loss=['mae'],optimizer=optimizer)   
                
        recover_model.fit(trainX, trainY, epochs=100,
                          batch_size = 10, verbose = 0)

        recover_test_seq  = trainX[-1:]
        recover_test_seq = recover_test_seq
        recover_test_seq = recover_test_seq.reshape(len(recover_test_seq), time_steps, 1)
        recover_preds = []
        
        for _ in range (DAYS_TO_PREDICT):
            recover_pred = recover_model.predict(recover_test_seq)
            recover_preds.append(recover_pred)
            recover_test_seq = np.roll(recover_test_seq,-1)
            recover_test_seq[-1] = recover_pred
            recover_test_seq = recover_test_seq.reshape(len(recover_test_seq ), time_steps, 1)

        

        return recover_preds

In [13]:

NN = Corona_recovered_model()
recovered_predictions = []
for i in range(len(Country_recovered)):
    result = NN.Model(recovered_df_scaled,i) 
    recovered_predictions.append(result)

In [14]:
recovered_predictions = (np.array(recovered_predictions).transpose())
recovered_predictions = recovered_predictions.reshape(((DAYS_TO_PREDICT),len(Country_recovered)))
recovered_predictions = sc.inverse_transform(recovered_predictions)
recovered_predictions = pd.DataFrame(recovered_predictions)
recovered_predictions = recovered_predictions.abs()
recovered_predictions['Dates'] = predict_recovered_index
recovered_predictions.set_index('Dates', inplace = True)
recovered_predictions.columns = Country_recovered
recovered_predictions.head()

,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,Azerbaijan,Bahamas,Bahrain,Bangladesh,Barbados,Belarus,Belgium,Belize,Benin,Bhutan,Bolivia,Bosnia and Herzegovina,Botswana,Brazil,Brunei,Bulgaria,Burkina Faso,Burma,Burundi,Cabo Verde,Cambodia,Cameroon,Canada,Central African Republic,Chad,Chile,China,Colombia,Comoros,Congo (Brazzaville),...,Senegal,Serbia,Seychelles,Sierra Leone,Singapore,Slovakia,Slovenia,Somalia,South Africa,South Sudan,Spain,Sri Lanka,Sudan,Suriname,Sweden,Switzerland,Syria,Taiwan*,Tajikistan,Tanzania,Thailand,Timor-Leste,Togo,Trinidad and Tobago,Tunisia,Turkey,US,Uganda,Ukraine,United Arab Emirates,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
Dates,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020/07/23,0.325757,88.781258,394.714264,0.005184,0.000442,0.001339,301.406097,1163.599365,34.500114,62.210922,478.424927,0.001727,516.525269,1217.413452,0.000510,390.335815,4.652790,0.000412,0.000789,0.000058,1893.311035,234.756363,0.000076,17644.789062,0.002049,103.725479,0.017897,0.436027,0.000433,30.967030,33.730743,0.012309,45.402832,1.003463,1.035081,4707.645508,11.441231,2321.191895,5.706834,0.000490,...,52.819923,0.123312,0.000144,4.613110,136.650467,0.020845,14.912605,1.463027,20055.158203,0.000574,0.879108,0.036505,0.018678,9.488503,0.0,0.186510,13.316520,0.000254,58.542721,0.000471,12.214292,0.000364,0.734151,0.000881,3.988669,1292.970947,19307.875000,46.246895,1243.597778,447.818054,0.013035,4.265934,40.249870,3.498982,18.309635,22.893633,0.002169,8.628717,0.000250,21.188618
2020/07/24,0.242739,102.552849,988.186829,0.006560,0.001674,0.000687,842.401733,807.005493,46.988594,61.088028,495.069427,0.002825,5519.978027,382.516449,0.000322,403.496094,1.348553,0.002476,0.000147,0.000043,390.183807,226.897598,0.000134,11504.543945,0.017057,53.725372,0.069350,0.680948,0.000610,15.905277,72.908638,0.083861,11.816132,0.430054,1.618824,8124.998047,10.327595,1688.992676,16.930948,0.000281,...,59.240959,0.038523,0.000129,8.164758,85.147522,0.016113,13.122049,8.586723,5224.019043,0.000589,1.683825,0.073516,0.039306,6.515016,0.0,0.196375,27.004129,0.000272,56.634640,0.000954,16.116768,0.000352,6.181478,0.000264,3.989340,1001.958374,16390.630859,20.466591,1238.642334,539.798584,0.013132,5.557353,103.871948,2.058056,19.333385,15.851160,0.003307,10.070937,0.000124,23.947062
2020/07/25,0.264000,80.435181,797.858643,0.006940,0.003741,0.000376,1551.837280,1249.378540,60.776112,61.367607,503.194916,0.004038,27591.796875,406.319519,0.000391,406.923645,0.608557,0.019102,0.000780,0.000047,598.524963,222.157867,0.000076,12390.920898,0.179120,44.340172,0.265230,9.087437,0.000060,22.168831,79.654510,0.555607,6.621246,0.144317,1.370425,16499.880859,10.308315,1327.394409,19.694996,0.000369,...,60.144913,0.096804,0.000131,1.880708,33.798119,0.016522,11.537640,18.834980,11665.834961,0.000589,2.420540,0.113554,0.060849,4.382408,0.0,0.181360,26.120264,0.000273,57.596931,0.001450,19.234192,0.000447,21.431856,0.003259,3.989294,1266.314209,13244.366211,3.554993,1227.058960,593.506287,0.013130,6.151987,58.132004,1.984788,31.788918,20.324543,0.003489,10.879076,0.000305,25.021353
2020/07/26,0.258548,114.714455,1237.934814,0.007046,0.009165,0.000800,6132.083984,691.761719,60.288132,61.297688,506.830750,0.004552,28875.447266,393.469299,0.000365,407.878448,2.412594,0.097389,0.000156,0.000046,328.199921,219.263229,0.000134,12716.021484,0.226495,39.727974,1.108801,7.008252,0.000364,13.818497,80.043259,0.424835,2.320312,0.094751,1.515619,25732.753906,10.307938,1081.022461,19.659361,0.000332,...,60.065487,0.056725,0.000130,3.858094,58.798859,0.016486,9.743791,8.393665,447.162750,0.000589,3.095105,0.156921,0.083352,4.054497,0.0,0.204218,26.383829,0.000273,57.140823,0.002188,22.406048,0.000669,21.288399,0.000571,3.989305,1195.473389,10503.265625,1.447978,1199.934937,618.229004,0.013129,6.430773,92.650261,1.92

In [15]:
Infected_Countries = pd.DataFrame(Country_recovered)
Infected_Countries.columns = ['Countries']
pd.set_option("max_rows", None)
Infected_Countries


,Countries
0,Afghanistan
1,Albania
2,Algeria
3,Andorra
4,Angola
5,Antigua and Barbuda
6,Argentina
7,Armenia
8,Australia
9,Austria


In [16]:
Country = input('Enter the name of the country to see the daily cases till now : ')

Enter the name of the country to see the daily cases till now : India


In [18]:
import plotly.graph_objects as go
fig = go.Figure()
fig.update_layout(template='plotly_dark',title={'text': 'Recovered Daily Cases','y':0.9,'x':0.5,'xanchor': 'center',
                                                'yanchor': 'top'},
                  xaxis_title="Dates",
                  yaxis_title="Number Of Cases")
fig.add_trace(go.Scatter(x=recovered_df.index, 
                         y=recovered_df[Country],
                         mode='lines+markers',
                         line=dict(color='green', width=2)))

In [19]:

fig = go.Figure()
fig.update_layout(template='plotly_dark',title={'text': 'Recovered Cases Predictions','y':0.9,'x':0.5,'xanchor': 'center',
                                                'yanchor': 'top'},xaxis_title="Dates",yaxis_title="Number Of Cases")
fig.update_xaxes(tickangle=90, showticklabels = True, type = 'category')
fig.add_trace(go.Scatter(x=recovered_predictions.index, 
                         y=recovered_predictions[Country],
                         mode='lines+markers',
                         line=dict(color='white', width=2)))